In [1]:
# Import Libraries
import cv2
import numpy as np
import time

print("Libraries imported successfully!")
print(f"OpenCV version: {cv2.__version__}")

Libraries imported successfully!
OpenCV version: 4.12.0


In [2]:
# Virtual boundary configuration (x, y, width, height)
BOUNDARY = {'x': 400, 'y': 150, 'width': 200, 'height': 300}

# Distance thresholds (in pixels)
DANGER_THRESHOLD = 30
WARNING_THRESHOLD = 100


In [3]:
# Camera resolution
CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480

# State colors (BGR format for OpenCV)
COLOR_SAFE = (0, 255, 0)      # Green
COLOR_WARNING = (0, 165, 255) # Orange
COLOR_DANGER = (0, 0, 255)    # Red
COLOR_BOUNDARY = (255, 255, 0) # Cyan
COLOR_HAND = (0, 255, 0)      # Green
COLOR_FINGERTIP = (0, 0, 255) # Red

print("Configuration loaded successfully!")

Configuration loaded successfully!


In [ ]:
#  Skin Detection Functions

def detect_skin_ycrcb(frame):
    """
    Detect skin using YCrCb color space (more robust than HSV/RGB)
    Args:
        frame: BGR image from camera 
    Returns:
        Binary mask where white pixels represent detected skin
     """
    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    
    # Define skin color range in YCrCb
    # Y: 0-255, Cr: 133-173, Cb: 77-127 (typical skin values)
    lower_skin = np.array([0, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)
    
    # Create binary mask
    mask = cv2.inRange(ycrcb, lower_skin, upper_skin)
    
    return mask

def detect_skin_hsv(frame):
    """
    Alternative: Detect skin using HSV color space
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define skin color range in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    return mask

print("Skin detection functions defined!")



Skin detection functions defined!


In [17]:
#  Image Processing Functions

def preprocess_mask(mask):
    """
    Clean up the skin mask using morphological operations
    
    Args:
        mask: Binary mask from skin detection
    
    Returns:
        Cleaned binary mask
    """
    # Create morphological kernels
    kernel_erode = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    kernel_dilate = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    # Erosion to remove noise
    mask = cv2.erode(mask, kernel_erode, iterations=2)
    
    # Dilation to fill gaps
    mask = cv2.dilate(mask, kernel_dilate, iterations=2)
    
    # Gaussian blur to smooth edges
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    
    return mask

def find_largest_contour(mask):
    """
    Find the largest contour in the mask (assumed to be the hand)
    Filter by position to avoid detecting face
    """
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    # Filter contours by position (assume hand is in lower half or sides)
    h, w = mask.shape
    valid_contours = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 1000:  # Too small
            continue
        
        # Get contour center
        M = cv2.moments(contour)
        if M['m00'] == 0:
            continue
        
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        
        # Filter: prefer contours in lower 2/3 of frame or on sides
        # This helps avoid detecting face which is usually in upper-middle
        if cy > h * 0.3 or cx < w * 0.3 or cx > w * 0.7:
            valid_contours.append((contour, area))
    
    if len(valid_contours) == 0:
        return None
    
    # Return largest valid contour
    largest_contour = max(valid_contours, key=lambda x: x[1])[0]
    
    return largest_contour

print("Image processing functions defined!")

#  Motion Detection Functions
prev_frame = None

def detect_hand_with_motion(frame, skin_mask):
    """
    Combine skin detection with motion detection to focus on moving hand
    """
    global prev_frame
    
    if prev_frame is None:
        prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return skin_mask
    
    # Convert current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate frame difference (motion)
    frame_diff = cv2.absdiff(prev_frame, gray)
    _, motion_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
    
    # Combine skin mask with motion mask
    combined_mask = cv2.bitwise_and(skin_mask, motion_mask)
    
    # Update previous frame
    prev_frame = gray.copy()
    
    return combined_mask
print("Motion detection functions defined!")



Image processing functions defined!
Motion detection functions defined!


In [7]:
#  Distance Calculation and State Classification

def calculate_distance_to_boundary(point, boundary):
    """
    Calculate minimum distance from a point to the virtual boundary
    
    Args:
        point: Tuple (x, y) representing the point
        boundary: Dictionary with keys x, y, width, height
    
    Returns:
        Distance in pixels
    """
    if point is None:
        return float('inf')
    
    px, py = point
    bx, by = boundary['x'], boundary['y']
    bw, bh = boundary['width'], boundary['height']
    
    # Calculate distance to each edge
    # If point is inside boundary, distance to closest edge
    # If point is outside, distance to closest corner or edge
    
    # Horizontal distance
    if px < bx:
        dx = bx - px
    elif px > bx + bw:
        dx = px - (bx + bw)
    else:
        dx = 0
    
    # Vertical distance
    if py < by:
        dy = by - py
    elif py > by + bh:
        dy = py - (by + bh)
    else:
        dy = 0
    
    # Euclidean distance
    distance = np.sqrt(dx**2 + dy**2)
    
    return distance

def classify_state(distance):
    """
    Classify interaction state based on distance
    
    Args:
        distance: Distance to boundary in pixels
    
    Returns:
        State string: 'SAFE', 'WARNING', or 'DANGER'
    """
    if distance < DANGER_THRESHOLD:
        return 'DANGER'
    elif distance < WARNING_THRESHOLD:
        return 'WARNING'
    else:
        return 'SAFE'

print("Distance and state functions defined!")



Distance and state functions defined!


In [8]:
#  Visualization Functions

def draw_boundary(frame, boundary):
    """
    Draw the virtual boundary on the frame
    """
    x, y = boundary['x'], boundary['y']
    w, h = boundary['width'], boundary['height']
    
    # Draw filled rectangle with transparency
    overlay = frame.copy()
    cv2.rectangle(overlay, (x, y), (x + w, y + h), COLOR_BOUNDARY, -1)
    cv2.addWeighted(overlay, 0.1, frame, 0.9, 0, frame)
    
    # Draw boundary outline
    cv2.rectangle(frame, (x, y), (x + w, y + h), COLOR_BOUNDARY, 3)
    
    # Draw boundary label
    cv2.putText(frame, "VIRTUAL BOUNDARY", (x, y - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_BOUNDARY, 2)

def draw_hand_features(frame, features):
    """
    Draw hand contour, hull, center, and fingertip
    """
    if features is None:
        return
    
    # Draw convex hull
    cv2.drawContours(frame, [features['hull']], 0, COLOR_HAND, 2)
    
    # Draw center point
    cv2.circle(frame, features['center'], 5, (255, 0, 255), -1)
    
    # Draw fingertip
    cv2.circle(frame, features['fingertip'], 8, COLOR_FINGERTIP, -1)
    cv2.circle(frame, features['fingertip'], 12, COLOR_FINGERTIP, 2)
    
    # Draw line from fingertip to boundary center
    boundary_center = (
        BOUNDARY['x'] + BOUNDARY['width'] // 2,
        BOUNDARY['y'] + BOUNDARY['height'] // 2
    )
    cv2.line(frame, features['fingertip'], boundary_center, (0, 255, 255), 2)

def draw_state_overlay(frame, state, distance, fps):
    """
    Draw state information overlay
    """
    # State color mapping
    state_colors = {
        'SAFE': COLOR_SAFE,
        'WARNING': COLOR_WARNING,
        'DANGER': COLOR_DANGER
    }
    
    color = state_colors.get(state, (128, 128, 128))
    
    # Draw state box
    cv2.rectangle(frame, (10, 10), (220, 100), color, -1)
    cv2.rectangle(frame, (10, 10), (220, 100), (255, 255, 255), 2)
    
    # Draw state text
    cv2.putText(frame, state, (25, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)
    cv2.putText(frame, f"Dist: {int(distance)}px", (25, 85), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    
    # Draw FPS counter
    cv2.rectangle(frame, (10, 110), (150, 150), (50, 50, 50), -1)
    cv2.rectangle(frame, (10, 110), (150, 150), (255, 255, 255), 2)
    cv2.putText(frame, f"FPS: {fps}", (25, 137), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # DANGER warning overlay
    if state == 'DANGER':
        overlay = frame.copy()
        h, w = frame.shape[:2]
        cv2.rectangle(overlay, (0, h//2 - 60), (w, h//2 + 60), COLOR_DANGER, -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        
        # Draw DANGER text
        cv2.putText(frame, "DANGER DANGER", (w//2 - 250, h//2 + 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255, 255, 255), 5)

def draw_info_panel(frame):
    """
    Draw information panel with instructions
    """
    h, w = frame.shape[:2]
    
    # Draw semi-transparent background
    overlay = frame.copy()
    cv2.rectangle(overlay, (w - 250, 10), (w - 10, 200), (40, 40, 40), -1)
    cv2.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
    
    # Draw border
    cv2.rectangle(frame, (w - 250, 10), (w - 10, 200), (200, 200, 200), 2)
    
    # Draw text
    texts = [
        "INSTRUCTIONS:",
        "Move hand toward",
        "cyan boundary",
        "",
        "Q - Quit",
        "R - Reset",
        "S - Screenshot"
    ]
    
    y_offset = 35
    for i, text in enumerate(texts):
        cv2.putText(frame, text, (w - 235, y_offset + i * 25), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

print("Visualization functions defined!")


Visualization functions defined!


In [ ]:
#  Main Processing Loop

def run_hand_tracking():
    """
    Main function to run the hand tracking danger detection system
    """
    # Initialize camera
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)
    
    if not cap.isOpened():
        print("Error: Could not open camera!")
        return
    
    print("Camera initialized successfully!")
    print("Controls:")
    print("  Q - Quit")
    print("  R - Reset")
    print("  S - Save screenshot")
    print("\nMove your hand toward the cyan boundary to trigger warnings...")
    
    # FPS calculation variables
    fps = 0
    frame_count = 0
    start_time = time.time()
    
    while True:
        # Capture frame
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame!")
            break
        
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Step 1: Detect skin
        skin_mask = detect_skin_ycrcb(frame)
        skin_mask = detect_hand_with_motion(frame, skin_mask)  # Add motion filtering
        
        # Step 2: Preprocess mask
        clean_mask = preprocess_mask(skin_mask)
        
        # Step 3: Find hand contour
        hand_contour = find_largest_contour(clean_mask)
        
        # Step 4: Extract hand features
        hand_features = get_hand_features(hand_contour)
        
        # Step 5: Calculate distance and classify state
        if hand_features:
            fingertip = hand_features['fingertip']
            distance = calculate_distance_to_boundary(fingertip, BOUNDARY)
            state = classify_state(distance)
        else:
            distance = float('inf')
            state = 'NO HAND DETECTED'
        
        # Step 6: Visualization
        draw_boundary(frame, BOUNDARY)
        draw_hand_features(frame, hand_features)
        draw_state_overlay(frame, state, distance, fps)
        draw_info_panel(frame)
        
        # Calculate FPS
        frame_count += 1
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = int(frame_count / elapsed_time)
            frame_count = 0
            start_time = time.time()
        
        # Display frame
        cv2.imshow('Hand Tracking Danger Detection', frame)
        
        # Optional: Show skin mask for debugging
        # cv2.imshow('Skin Mask', clean_mask)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # Q or ESC to quit
            print("Exiting...")
            break
        elif key == ord('r'):  # R to reset
            print("Reset...")
        elif key == ord('s'):  # S to save screenshot
            filename = f"screenshot_{int(time.time())}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Screenshot saved: {filename}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    print("Camera released and windows closed.")

print("Main function defined!")
print("\nReady to run! ")



Main function defined!

Ready to run! 


In [ ]:
#  Run the System

run_hand_tracking()

Camera initialized successfully!
Controls:
  Q - Quit
  R - Reset
  S - Save screenshot

Move your hand toward the cyan boundary to trigger warnings...
Exiting...
Camera released and windows closed.
